# Conjunto de dados

aXXX

# Processamento dos dados

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.0 MB/s eta 0:00:00


In [ ]:
pip install dalex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dalex: filename=dalex-1.7.2-py3-none-any.whl size=1042898 sha256=3e8bcac5ff0451658c400531019961e11f343a1017e4281483ad1b2fc6d89cdf
  Stored in directory: /root/.cache/pip/wheels/cd/5e/ec/848120d5b913158656c0f8ef316233dd3356effa64aef19fba
Successfully built dalex


In [ ]:
!pip install aif360 numpy pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 5.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px



from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score
from sklearn import metrics

from io import StringIO
import sys

import re
import sys
from io import StringIO

import dalex as dx

In [ ]:
diabetes = pd.read_csv("diabetes_012_health_indicators_BRFSS2021.csv")

In [ ]:
diabetes.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,0,1.0,1,15.0,1.0,0.0,0.0,0,1,...,1,0.0,5.0,10.0,20.0,0.0,0,11,4.0,5.0
1,2.0,1,0.0,1,28.0,0.0,0.0,1.0,0,1,...,1,0.0,2.0,0.0,0.0,0.0,0,11,4.0,3.0
2,2.0,1,1.0,1,33.0,0.0,0.0,0.0,1,1,...,1,0.0,2.0,10.0,0.0,0.0,0,9,4.0,7.0
3,2.0,0,1.0,1,29.0,0.0,1.0,1.0,1,1,...,1,0.0,5.0,0.0,30.0,1.0,1,12,3.0,4.0
4,0.0,0,0.0,1,24.0,1.0,0.0,0.0,0,0,...,1,0.0,3.0,0.0,0.0,1.0,1,13,5.0,6.0


In [ ]:
diabetes.shape

(236378, 22)

In [ ]:
diabetes = diabetes[['Diabetes_012','HighBP', 'HighChol', 'BMI', 'Smoker', 'HeartDiseaseorAttack',
            'PhysActivity', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age', 'Income', 'Sex']]

In [ ]:
df_amostra = diabetes.sample(frac=0.10, random_state=42)  # Usa 10% dos dados

In [ ]:
df_amostra.shape

(23638, 13)

In [ ]:
diabetes = df_amostra

In [ ]:
diabetes.shape

(23638, 13)

In [ ]:
diabetes['Diabetes_012'].value_counts()

,count
Diabetes_012,
0.0,19729
2.0,3391
1.0,518


In [ ]:
# Para classificação binária, combinaremos pré-diabetes e diabetes (classes 1 e 2)
diabetes['Diabetes_binary'] = diabetes['Diabetes_012'].apply(lambda x: 1 if x >= 1 else 0)

In [ ]:
diabetes['Diabetes_binary'].value_counts()

,count
Diabetes_binary,
0,19729
1,3909


In [ ]:
diabetes.dtypes

,0
Diabetes_012,float64
HighBP,int64
HighChol,float64
BMI,float64
Smoker,float64
HeartDiseaseorAttack,float64
PhysActivity,int64
GenHlth,float64
MentHlth,float64
PhysHlth,float64


In [ ]:
diabetes.columns

Index(['Diabetes_012', 'HighBP', 'HighChol', 'BMI', 'Smoker',
       'HeartDiseaseorAttack', 'PhysActivity', 'GenHlth', 'MentHlth',
       'PhysHlth', 'Age', 'Income', 'Sex', 'Diabetes_binary'],
      dtype='object')

In [ ]:
diabetes.Sex.value_counts()

,count
Sex,
0,12354
1,11284


# Primeira etapa do experimento

* dataset do compas
* vários algoritmos
* várias métricas de **fairness**

Fairness by unawareness ou justiça através do desconhecimento é um conceito ingênuo de  justiça em que um algoritmo justo é definido quando não usa nenhum atributo protegido no processo de treinamento. Então não existe essa ideia de treinar sem o atributo protegido.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(["Diabetes_binary"], axis =1),
                                                    diabetes.Diabetes_binary,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
diabetes.describe()

,Diabetes_012,HighBP,HighChol,BMI,Smoker,HeartDiseaseorAttack,PhysActivity,GenHlth,MentHlth,PhysHlth,Age,Income,Sex,Diabetes_binary
count,23638.000000,23638.000000,23638.000000,23638.000000,23638.000000,23638.000000,23638.000000,23638.000000,23638.000000,23638.000000,23638.000000,23638.000000,23638.000000,23638.000000
mean,0.308825,0.421102,0.398469,28.888104,0.408326,0.086767,0.780565,2.470810,3.900626,3.748794,7.909045,6.925332,0.477367,0.165369
std,0.707378,0.493746,0.489593,6.524160,0.491534,0.281499,0.413872,1.026314,7.837531,8.218224,3.228703,2.375586,0.499498,0.371521
min,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,24.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,5.000000,5.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,28.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,8.000000,7.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,32.000000,1.000000,0.000000,1.000000,3.000000,3.000000,2.000000,10.000000,9.000000,1.000000,0.000000
max,2.000000,1.000000,1.000000,95.000000,1.000000,1.000000,1.000000,5.000000,30.000000,30.000000,13.000000,11.000000,1.000000,1.000000


In [ ]:
categorical_features = ['Sex']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_features = ['HighBP', 'HighChol', 'BMI', 'Smoker', 'HeartDiseaseorAttack',
       'PhysActivity', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age', 'Income'
       ]

numerical_transformer = Pipeline(steps=[
    ('scale', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
])

### Treinando para vários algoritmos

In [ ]:
  clf_forest = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', RandomForestClassifier(n_estimators=200, max_depth=7, random_state=123))
  ])

  clf_logreg = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LogisticRegression())
  ])

  clf_gb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', GradientBoostingClassifier(random_state=123))
  ])

  clf_xgb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', XGBClassifier(random_state=123))
  ])

  clf_lgbm = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LGBMClassifier(random_state=123))
  ])

  clf_catboost = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', CatBoostClassifier(random_state=123, verbose=0))
  ])

  clf_ridge = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', RidgeClassifier())
  ])

  clf_sgd = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', SGDClassifier(random_state=123))
  ])

  clf_svc = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', SVC(random_state=123))
  ])

  clf_knn = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', KNeighborsClassifier())
  ])

  clf_nb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', GaussianNB())
  ])

  clf_mlp = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', MLPClassifier(random_state=123))
  ])

In [ ]:
model_pipelines = {
    'Random Forest': clf_forest,
    'Logistic Regression': clf_logreg,
    'Gradient Boosting': clf_gb,
    'XGBoost': clf_xgb,
    #'LightGBM': clf_lgbm,
    'CatBoost': clf_catboost,
    'Ridge Classifier': clf_ridge,
    'SGD Classifier': clf_sgd,
    'SVC': clf_svc,
    'KNeighbors': clf_knn,
    'GaussianNB': clf_nb,
    'MLPClassifier': clf_mlp
}

In [ ]:
trained_models = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Avaliando para várias métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models.items():
    y_pred = model.predict(X_test)
    relatorio = {'accuracy': accuracy_score(y_test, y_pred), 'model': model_name}
    resultados.append(relatorio)

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

    accuracy                model
0   0.836999        Random Forest
1   0.835730  Logistic Regression
2   0.839114    Gradient Boosting
3   0.832487              XGBoost
4   0.836999             CatBoost
5   0.837281     Ridge Classifier
6   0.836999       SGD Classifier
7   0.836999                  SVC
8   0.819092           KNeighbors
9   0.791032           GaussianNB
10  0.836717        MLPClassifier


In [ ]:
resultados = []
for model_name, model in trained_models.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

    accuracy    recall  precision       auc        f1                model
0   0.836999  0.083898   0.568966  0.535606  0.146233        Random Forest
1   0.835730  0.174576   0.518892  0.571135  0.261256  Logistic Regression
2   0.839114  0.189831   0.547677  0.579269  0.281938    Gradient Boosting
3   0.832487  0.211864   0.492126  0.584112  0.296209              XGBoost
4   0.836999  0.189831   0.528302  0.578001  0.279302             CatBoost
5   0.837281  0.076271   0.584416  0.532723  0.134933     Ridge Classifier
6   0.836999  0.117797   0.547244  0.549172  0.193863       SGD Classifier
7   0.836999  0.105932   0.553097  0.544424  0.177809                  SVC
8   0.819092  0.238136   0.422556  0.586591  0.304607           KNeighbors
9   0.791032  0.444915   0.388314  0.652515  0.414692           GaussianNB
10  0.836717  0.213559   0.522822  0.587328  0.303249        MLPClassifier


In [ ]:
#df_resultados.to_excel(r"Performance padrão dos algoritmos sem balanceamento.xlsx", index=False)

### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers = {}

for model_name, model in trained_models.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 7092 rows 13 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 7092 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7ff8f89dfba0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0128, mean = 0.163, max = 0.74
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.67, mean = 0.00362, max = 0.981
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

Para automatizar o processo de concatenar os resultados de desempenho de todos os modelos explicados com DALEX, irei iterar sobre os explicadores armazenados em um dicionário e concatenar seus resultados.

In [ ]:
performance_results = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name
    performance_results.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results = pd.concat(performance_results, ignore_index=True)

o método model_perfomance realiza o cálculo de medidas de desempenho do modelo


In [ ]:
protected = X_test.Sex

In [ ]:
model_fairness_results = {}

for model_name, explainer in explainers.items():
    fairness_result = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results[model_name] = fairness_result

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
fairness_checks = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result in model_fairness_results.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks = pd.DataFrame(fairness_checks)

# Exibir o DataFrame com os resultados
print(all_fairness_checks)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 2 metrics: TPR, FPR

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR   STP
0  1.333333  1.025424  1.214844  0.733333  0.96
Métricas de viés detectadas para Random Forest: TPR=1.333333, ACC=1.025424, PPV=1.214844, FPR=0.733333, STP=0.96
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC    PPV       FPR       STP


Este código captura e processa a saída do método fairness_check() para cada modelo, extrai as métricas relevantes e armazena essas informações em um DataFrame para análise. Isso facilita a visualização e comparação dos resultados de diferentes modelos em termos de métricas de viés detectadas.

In [ ]:
all_fairness_checks.head()

,Modelo,Num Biases Detected,TPR,ACC,PPV,FPR,STP
0,Random Forest,2.0,1.333333,1.025424,1.214844,0.733333,0.960000
1,Logistic Regression,2.0,0.886486,1.024242,1.084000,0.666667,0.738462
2,Gradient Boosting,2.0,0.958763,1.026570,1.125483,0.657895,0.784615
3,XGBoost,NaN,1.014286,1.020631,1.016393,0.872340,0.906667
4,CatBoost,NaN,0.928934,1.020531,1.042471,0.743590,0.818182


# Segunda etapa do experimento - Balancear os dados para mitigar o viés

* dataset do compas
* balanceamento dos dados no treinamento
* várias técnicas de balanceamento dos dados
* vários algoritmos
* várias métricas de **fairness**

### Abordagem 1 - Embaralhar a coluna de sexo desbalanceada

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(["Diabetes_binary", "Diabetes_012"], axis =1),
                                                    diabetes.Diabetes_binary,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
X_train_abordagemUm = X_train.copy().reset_index(drop=True)

X_train_abordagemUm["Sex"] = X_train["Sex"].sample(frac=1).reset_index(drop=True)

#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_abordagemUm, y_train)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemUm = pd.DataFrame(resultados)
print(df_resultados_abordagemUm)


    accuracy    recall  precision       auc        f1                model
0   0.837140  0.085593   0.570621  0.536369  0.148858        Random Forest
1   0.836435  0.177966   0.525000  0.572914  0.265823  Logistic Regression
2   0.839114  0.188136   0.548148  0.578591  0.280126    Gradient Boosting
3   0.832487  0.203390   0.491803  0.580721  0.287770              XGBoost
4   0.836858  0.191525   0.526807  0.578594  0.280920             CatBoost
5   0.837563  0.076271   0.592105  0.532892  0.135135     Ridge Classifier
6   0.835871  0.100847   0.536036  0.541713  0.169757       SGD Classifier
7   0.838832  0.110169   0.582960  0.547219  0.185317                  SVC
8   0.827129  0.244915   0.463141  0.594125  0.320399           KNeighbors
9   0.787789  0.448305   0.382502  0.651927  0.412798           GaussianNB
10  0.835448  0.206780   0.513684  0.583853  0.294864        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 7092 rows 12 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 7092 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7ff8f89dfba0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0135, mean = 0.163, max = 0.71
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.7, mean = 0.00343, max = 0.982
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data       

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.Sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 2 metrics: TPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  1.478261  1.021739  1.074954  1.166667  1.272727
Métricas de viés detectadas para Random Forest: TPR=1.478261, ACC=1.021739, PPV=1.074954, FPR=1.166667, STP=1.272727
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 1 metric: TPR

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot be automatically approved based on these metrics.

Ratios of metrics, based on '1'. Parameter 'epsilon'

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



In [ ]:
fairness_result_balanceamento.plot()

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 2 - Duplicar os dados de treinamento com a classe oposta

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(["Diabetes_binary",'Diabetes_012'], axis =1),
                                                    diabetes.Diabetes_binary,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
df2 = X_train.copy()

df2["Sex"] = df2["Sex"].replace({1:0, 0: 1})
X_train2 = pd.concat([df2, X_train])
print(X_train2.shape, X_train.shape)
X_train2['Sex'].value_counts()

(33092, 12) (16546, 12)


,count
Sex,
1,16546
0,16546


In [ ]:
y_train2 = np.concatenate([y_train, y_train])

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados duplicados no treinamento com o sexo (classe) oposto
trained_models_duplicados = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train2, y_train2)
    trained_models_duplicados[model_name] = model
    print(f"Treinando {model_name} com dados de treinamento duplicados")

Treinando Random Forest com dados de treinamento duplicados
Treinando Logistic Regression com dados de treinamento duplicados
Treinando Gradient Boosting com dados de treinamento duplicados
Treinando XGBoost com dados de treinamento duplicados
Treinando CatBoost com dados de treinamento duplicados
Treinando Ridge Classifier com dados de treinamento duplicados
Treinando SGD Classifier com dados de treinamento duplicados
Treinando SVC com dados de treinamento duplicados
Treinando KNeighbors com dados de treinamento duplicados
Treinando GaussianNB com dados de treinamento duplicados
Treinando MLPClassifier com dados de treinamento duplicados


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_duplicados.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemDois = pd.DataFrame(resultados)
print(df_resultados_abordagemDois)


    accuracy    recall  precision       auc        f1                model
0   0.836999  0.083051   0.569767  0.535267  0.144970        Random Forest
1   0.836576  0.177119   0.526448  0.572659  0.265060  Logistic Regression
2   0.840525  0.192373   0.560494  0.581132  0.286435    Gradient Boosting
3   0.831077  0.210169   0.482490  0.582588  0.292798              XGBoost
4   0.835025  0.195763   0.511062  0.579191  0.283088             CatBoost
5   0.837704  0.076271   0.596026  0.532977  0.135237     Ridge Classifier
6   0.833756  0.000847   1.000000  0.500424  0.001693       SGD Classifier
7   0.837845  0.122034   0.558140  0.551376  0.200278                  SVC
8   0.817118  0.241525   0.414847  0.586764  0.305303           KNeighbors
9   0.787789  0.446610   0.382161  0.651248  0.411880           GaussianNB
10  0.832910  0.230508   0.495446  0.591827  0.314633        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_dois = {}

for model_name, model in trained_models_duplicados.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_dois[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 7092 rows 12 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 7092 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7ff8f89dfba0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0148, mean = 0.163, max = 0.723
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.692, mean = 0.00335, max = 0.982
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data    

In [ ]:
performance_results_balanceamento_dois = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_dois.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_dois.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_dois = pd.concat(performance_results_balanceamento_dois, ignore_index=True)

In [ ]:
protected = X_test.Sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_dois = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_dois.items():
    fairness_result_balanceamento_dois = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento_dois[model_name] = fairness_result_balanceamento_dois

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_dois = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_dois in model_fairness_results_balanceamento_dois.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_dois is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_dois.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_dois.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_dois = pd.DataFrame(fairness_checks_balanceamento_dois)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_dois)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 1 metric: TPR

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot be automatically approved based on these metrics.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  1.530303  1.025424  1.227273  0.857143  1.130435
Métricas de viés detectadas para Random Forest: TPR=1.530303, ACC=1.025424, PPV=1.227273, FPR=0.857143, STP=1.130435
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 1 metric: TPR

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot

In [ ]:
# Gráfico de Barras: Comparação do Número de Métricas de Viés Detectadas por Modelo
fig = px.bar(
    all_fairness_checks_balanceamento_dois,
    x='Modelo',
    y='Num Biases Detected',
    title="Número de Métricas de Viés Detectadas por Modelo",
    color='Num Biases Detected',  # Colore as barras com base no valor
    color_continuous_scale='viridis',
    labels={"Num Biases Detected": "Número de Métricas de Viés", "Modelo": "Modelo"}
)

# Ajuste de layout para legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    title_x=0.5,  # Centraliza o título
    yaxis_title="Número de Métricas de Viés",
    xaxis_title="Modelo"
)

# Exibindo o gráfico
fig.show()

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_dois.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_dois in model_fairness_results_balanceamento_dois.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_dois.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression

Found NaN's or 0's for models: {'LogisticRegression'}
It is advisable to check 'metric_ratios'


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting

Found NaN's or 0's for models: {'GradientBoostingClassifier'}
It is advisable to check 'metric_ratios'


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 3 - Subamostragem da Classe Majoritária

In [ ]:
diabetes.head()

,Diabetes_012,HighBP,HighChol,BMI,Smoker,HeartDiseaseorAttack,PhysActivity,GenHlth,MentHlth,PhysHlth,Age,Income,Sex,Diabetes_binary
111866,2.0,1,1.0,32.0,1.0,1.0,1,2.0,15.0,0.0,7,6.0,1,1
206199,0.0,0,1.0,28.0,0.0,0.0,0,4.0,4.0,30.0,10,7.0,0,0
144211,0.0,0,1.0,24.0,1.0,0.0,1,2.0,0.0,0.0,12,6.0,0,0
89872,2.0,1,1.0,38.0,0.0,0.0,1,3.0,0.0,0.0,7,8.0,1,1
34953,0.0,0,1.0,30.0,0.0,0.0,0,4.0,10.0,0.0,5,7.0,0,0


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(["Diabetes_binary", 'Diabetes_012'], axis=1),
                                                    diabetes.Diabetes_binary,
                                                    test_size=0.3,
                                                    random_state=42)

# Aplicar subamostragem da classe majoritária
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados subamostrados
trained_models_subamostragem = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_res, y_train_res)
    trained_models_subamostragem[model_name] = model
    print(f"Treinado {model_name} com subamostragem")

Treinado Random Forest com subamostragem
Treinado Logistic Regression com subamostragem
Treinado Gradient Boosting com subamostragem
Treinado XGBoost com subamostragem
Treinado CatBoost com subamostragem
Treinado Ridge Classifier com subamostragem
Treinado SGD Classifier com subamostragem
Treinado SVC com subamostragem
Treinado KNeighbors com subamostragem
Treinado GaussianNB com subamostragem
Treinado MLPClassifier com subamostragem


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_subamostragem.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemTres = pd.DataFrame(resultados)
print(df_resultados_abordagemTres)


    accuracy    recall  precision       auc        f1                model
0   0.700226  0.799470   0.322753  0.740423  0.459858        Random Forest
1   0.718556  0.759717   0.332817  0.735228  0.462863  Logistic Regression
2   0.703892  0.782686   0.323358  0.735806  0.457645    Gradient Boosting
3   0.682459  0.742049   0.300000  0.706595  0.427263              XGBoost
4   0.699239  0.782686   0.319510  0.733037  0.453777             CatBoost
5   0.714467  0.761484   0.329385  0.733511  0.459856     Ridge Classifier
6   0.685279  0.725265   0.299416  0.701475  0.423851       SGD Classifier
7   0.689086  0.801237   0.314167  0.734511  0.451356                  SVC
8   0.666808  0.741166   0.288415  0.696925  0.415244           KNeighbors
9   0.746193  0.655477   0.344796  0.709450  0.451888           GaussianNB
10  0.705302  0.759717   0.321135  0.727342  0.451444        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_tres = {}

for model_name, model in trained_models_subamostragem.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_tres[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 7092 rows 12 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 7092 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7ff8f89dfba0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0474, mean = 0.42, max = 0.914
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.914, mean = -0.261, max = 0.947
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento_tres = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_tres.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_tres.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_tres = pd.concat(performance_results_balanceamento_tres, ignore_index=True)

In [ ]:
protected = X_test.Sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_tres = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_tres.items():
    fairness_result_balanceamento_tres = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento_tres[model_name] = fairness_result_balanceamento_tres

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_tres = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_tres in model_fairness_results_balanceamento_tres.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_tres is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_tres.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_tres.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_tres = pd.DataFrame(fairness_checks_balanceamento_tres)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_tres)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR      STP
0  0.969212  1.024602  0.975535  0.918919  0.92233
Métricas de viés detectadas para Random Forest: TPR=0.969212, ACC=1.024602, PPV=0.975535, FPR=0.918919, STP=0.92233
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  0.898876  1.036879  0.976261  0.850318  0.853165
Métricas de viés detectadas para Log

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_tres.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_tres in model_fairness_results_balanceamento_tres.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_tres.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 4 - Utilização da biblioteca SMOTE para criar exemplos sintéticos da classe minoritária - KMeans-SMOTE

In [ ]:
diabetes.head()

,Diabetes_012,HighBP,HighChol,BMI,Smoker,HeartDiseaseorAttack,PhysActivity,GenHlth,MentHlth,PhysHlth,Age,Income,Sex,Diabetes_binary
111866,2.0,1,1.0,32.0,1.0,1.0,1,2.0,15.0,0.0,7,6.0,1,1
206199,0.0,0,1.0,28.0,0.0,0.0,0,4.0,4.0,30.0,10,7.0,0,0
144211,0.0,0,1.0,24.0,1.0,0.0,1,2.0,0.0,0.0,12,6.0,0,0
89872,2.0,1,1.0,38.0,0.0,0.0,1,3.0,0.0,0.0,7,8.0,1,1
34953,0.0,0,1.0,30.0,0.0,0.0,0,4.0,10.0,0.0,5,7.0,0,0


In [ ]:
diabetes.columns

Index(['Diabetes_012', 'HighBP', 'HighChol', 'BMI', 'Smoker',
       'HeartDiseaseorAttack', 'PhysActivity', 'GenHlth', 'MentHlth',
       'PhysHlth', 'Age', 'Income', 'Sex', 'Diabetes_binary'],
      dtype='object')

In [ ]:
# Supomos que 'compas' seja seu DataFrame original
categorical_columns = [
       ]
non_categorical_columns = diabetes.drop(columns=categorical_columns)  # Selecionar as colunas restantes

# Inicializar o OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

# Aplicar o encoder nas colunas categóricas
encoded_data = encoder.fit_transform(diabetes[categorical_columns]).toarray()  # Convertendo para array denso

# Criar um DataFrame com as colunas codificadas
encoded_df = pd.DataFrame(encoded_data,
                          columns=encoder.get_feature_names_out(categorical_columns),
                          index=diabetes.index)  # Preserva os índices originais

# Concatenar o DataFrame codificado com as colunas não categóricas
df_encoded = pd.concat([non_categorical_columns, encoded_df], axis=1)

# Exibir o DataFrame final
df_encoded.head()

,Diabetes_012,HighBP,HighChol,BMI,Smoker,HeartDiseaseorAttack,PhysActivity,GenHlth,MentHlth,PhysHlth,Age,Income,Sex,Diabetes_binary
111866,2.0,1,1.0,32.0,1.0,1.0,1,2.0,15.0,0.0,7,6.0,1,1
206199,0.0,0,1.0,28.0,0.0,0.0,0,4.0,4.0,30.0,10,7.0,0,0
144211,0.0,0,1.0,24.0,1.0,0.0,1,2.0,0.0,0.0,12,6.0,0,0
89872,2.0,1,1.0,38.0,0.0,0.0,1,3.0,0.0,0.0,7,8.0,1,1
34953,0.0,0,1.0,30.0,0.0,0.0,0,4.0,10.0,0.0,5,7.0,0,0


In [ ]:
diabetes_encoded = df_encoded

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(diabetes_encoded.drop(["Diabetes_012", 'Diabetes_binary'], axis=1),
                                                    diabetes_encoded.Diabetes_binary,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['Sex'].astype(str) + "_" + y_train.astype(str)

# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_group = smote.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_smote['Sex'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_smote['Sex'].value_counts())


1
0    14628
1    14628
Name: count, dtype: int64
Sex
1    14628
0    14628
Name: count, dtype: int64


O SMOTE cria exemplos sintéticos da classe minoritária, aumentando o conjunto de dados de forma mais natural.

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados subamostrados
trained_models_smote = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_smote, y_income)
    trained_models_smote[model_name] = model
    print(f"Treinando {model_name} com smote")

Treinando Random Forest com smote
Treinando Logistic Regression com smote
Treinando Gradient Boosting com smote
Treinando XGBoost com smote
Treinando CatBoost com smote
Treinando Ridge Classifier com smote
Treinando SGD Classifier com smote
Treinando SVC com smote
Treinando KNeighbors com smote
Treinando GaussianNB com smote
Treinando MLPClassifier com smote


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_smote.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemQuatro = pd.DataFrame(resultados)
print(df_resultados_abordagemQuatro)


    accuracy    recall  precision       auc        f1                model
0   0.790891  0.573322   0.393572  0.702768  0.466739        Random Forest
1   0.722081  0.733216   0.332133  0.726591  0.457174  Logistic Regression
2   0.818810  0.428445   0.431879  0.660699  0.430155    Gradient Boosting
3   0.827975  0.293286   0.441489  0.611408  0.352442              XGBoost
4   0.833615  0.272968   0.463964  0.606534  0.343715             CatBoost
5   0.721094  0.729682   0.330665  0.724573  0.455096     Ridge Classifier
6   0.661873  0.791519   0.293002  0.714384  0.427685       SGD Classifier
7   0.788071  0.555654   0.386126  0.693934  0.455632                  SVC
8   0.727157  0.575088   0.309264  0.665564  0.402224           KNeighbors
9   0.743514  0.649293   0.340751  0.705351  0.446944           GaussianNB
10  0.791878  0.516784   0.386394  0.680456  0.442177        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_quatro = {}

for model_name, model in trained_models_smote.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_quatro[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 7092 rows 12 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 7092 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7ff8f89dfba0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0225, mean = 0.322, max = 0.845
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.845, mean = -0.162, max = 0.968
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento_quatro = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_quatro.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_quatro.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_quatro = pd.concat(performance_results_balanceamento_quatro, ignore_index=True)

In [ ]:
protected = X_test.Sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_quatro = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_quatro.items():
    fairness_result_balanceamento_quatro = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento_quatro[model_name] = fairness_result_balanceamento_quatro

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_quatro = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_quatro in model_fairness_results_balanceamento_quatro.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_quatro is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_quatro.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])

        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_quatro.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_quatro = pd.DataFrame(fairness_checks_balanceamento_quatro)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_quatro)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR  ACC       PPV       FPR      STP
0  1.017606  1.0  0.940887  1.030303  1.00431
Métricas de viés detectadas para Random Forest: TPR=1.017606, ACC=1.0, PPV=0.940887, FPR=1.030303, STP=0.862408
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  0.995918  0.970027  0.884181  1.093633  1.046512
Métricas de viés detectadas para Logistic Regressi

 graficos das metricas de fairness do dalex e avaliação para algunos modelos

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_quatro.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_quatro in model_fairness_results_balanceamento_quatro.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_quatro.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 5 - FairSmote (Implementação de um algoritmo que remove rótulos tendenciosos e rebalanceia as distribuições internas de forma que, com base no atributo sensível, os exemplos sejam iguais tanto nas classes positivas quanto negativas.)

In [ ]:
from __future__ import print_function, division
import pdb
import unittest
import random
from collections import Counter
from sklearn.neighbors import NearestNeighbors as NN

class smote(object):
  def __init__(self, pd_data, neighbor=5,r=2 ,up_to_num=[],auto=True):
    """
    :param pd_data: panda.DataFrame, the last column must be class label
    :param neighbor: num of nearst neighbors to select
    :param up_to_num: size of minorities to over-sampling
    :param up_to_max: if up_to_num is not supplied, all minority classes will
                      be over-sampled as much as majority class
    :return panda.DataFrame smoted data
    """
    self.set_data(pd_data)
    self.auto = auto
    self.neighbor = neighbor
    self.up_to_max = False
    self.up_to_num = up_to_num
    self.r = r
    self.label_num = len(set(pd_data[pd_data.columns[-1]].values))
    #if up_to_num:
    #  label_num = len(set(pd_data[pd_data.columns[-1]].values))
    #  if label_num - 1 != len(up_to_num):
    #    raise ValueError(
    #      "should set smoted size for " + str(label_num - 1) + " minorities")
    #  self.up_to_num = up_to_num
    #else:
    #  self.up_to_max = True

  def set_data(self, pd_data):
    if not pd_data.empty:# and isinstance(
        #pd_data.ix[:, pd_data.columns[-1]].values[0], str):
      self.data = pd_data
    else:
      raise ValueError(
        "The last column of pd_data should be string as class label")

  def get_majority_num(self):
    total_data = self.data.values.tolist()
    labelCont = Counter(self.data[self.data.columns[-1]].values)
    majority_num = max(labelCont.values())
    return majority_num

  def run(self):
    """
    run smote
    """

    def get_ngbr(data_no_label, knn):
      rand_sample_idx = random.randint(0, len(data_no_label) - 1)
      rand_sample = data_no_label[rand_sample_idx]
      distance, ngbr = knn.kneighbors(rand_sample.reshape(1, -1))
      # while True:
      rand_ngbr_idx = random.randint(0, len(ngbr))
      #   if distance[rand_ngbr_idx] == 0:
      #     continue  # avoid the sample itself, where distance ==0
      #   else:
      return data_no_label[rand_ngbr_idx], rand_sample

    total_data = self.data.values.tolist()
    labelCont = Counter(self.data[self.data.columns[-1]].values)
    majority_num = max(labelCont.values())
    for label, num in labelCont.items():
      if num < majority_num:
        to_add = majority_num - num
        last_column = self.data[self.data.columns[-1]]
        data_w_label = self.data.loc[last_column == label]
        data_no_label = data_w_label[self.data.columns[:-1]].values
        if len(data_no_label) < self.neighbor:
          num_neigh = len(data_no_label) # void # of neighbors >= sample size
        else:
          num_neigh = self.neighbor
        knn = NN(n_neighbors=num_neigh,p=self.r,algorithm='ball_tree').fit(data_no_label)
        if self.auto:
          to_add = to_add
        else:
          to_add = self.up_to_num
        for _ in range(to_add):
          rand_ngbr, sample = get_ngbr(data_no_label, knn)
          new_row = []
          for i, one in enumerate(rand_ngbr):
            gap = random.random()
            new_row.append(max(0, sample[i] + (
            sample[i] - one) * gap))  # here, feature vlaue should >=0
          new_row.append(label)
          total_data.append(new_row)
    return pd.DataFrame(total_data)

In [ ]:
def apply_smote(df):
    df.reset_index(drop=True,inplace=True)
    cols = df.columns
    smt = smote(df)
    df = smt.run()
    df.columns = cols
    return df

In [ ]:
dataset_orig_train, dataset_orig_test = train_test_split(diabetes_encoded, test_size=0.2, shuffle = True)

In [ ]:
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Diabetes_binary'], dataset_orig_train['Diabetes_binary']
X_test, y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Diabetes_binary'], dataset_orig_test['Diabetes_binary']

In [ ]:
train_df = X_train
train_df['Diabetes_binary'] = y_train

In [ ]:
train_df = apply_smote(train_df)

In [ ]:
y_train = train_df.Diabetes_binary
X_train = train_df.drop('Diabetes_binary', axis = 1)

#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors


TypeError: Sparse data was passed for X, but dense data is required. Use '.toarray()' to convert to a dense numpy array.

#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemCinco = pd.DataFrame(resultados)
print(df_resultados_abordagemCinco)


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.Sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


### Abordagem 6 - ADASYN

In [ ]:
from imblearn.over_sampling import ADASYN

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(diabetes_encoded.drop(["Diabetes_binary"], axis=1),
                                                    diabetes_encoded.Diabetes_binary,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['Sex'].astype(str) + "_" + y_train.astype(str)

# Aplicar ADASYN
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_group = adasyn.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_adasyn['Sex'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_adasyn['Sex'].value_counts())


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_adasyn, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.755246  0.749145   0.492831  0.753157  0.594540        Random Forest
1   0.731600  0.765385   0.463509  0.743172  0.577369  Logistic Regression
2   0.777152  0.750000   0.524350  0.767852  0.617197    Gradient Boosting
3   0.798342  0.719658   0.561708  0.771392  0.630948              XGBoost
4   0.798239  0.716667   0.561809  0.770300  0.629859             CatBoost
5   0.709182  0.752137   0.437702  0.723894  0.553372     Ridge Classifier
6   0.699969  0.797436   0.431645  0.733352  0.560108       SGD Classifier
7   0.739175  0.812393   0.474065  0.764253  0.598740                  SVC
8   0.735695  0.700855   0.465645  0.723762  0.559536           KNeighbors
9   0.792916  0.313248   0.637946  0.628626  0.420178           GaussianNB
10  0.762719  0.762393   0.503102  0.762607  0.606184        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 9769 rows 102 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9769 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7cef11bcc5e0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0138, mean = 0.405, max = 0.975
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.898, mean = -0.166, max = 0.962
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data    

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 3 metrics: PPV, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  0.852288  1.053908  0.477639  0.761194  0.591449
Métricas de viés detectadas para Random Forest: TPR=0.852288, ACC=1.053908, PPV=0.477639, FPR=0.761194, STP=0.591449
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 2 metrics: PPV, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV 

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 7 - Random Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(diabetes_encoded.drop(["Diabetes_binary"], axis=1),
                                                    diabetes_encoded.Diabetes_binary,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['Sex'].astype(str) + "_" + y_train.astype(str)

# Agora balanceia os grupos combinados (ex: mulher_<=50K, homem_>50K, etc.)
ros = RandomOverSampler(random_state=42)
X_train_ros, y_group = ros.fit_resample(X_train, group)

# Separar novamente income e sex
#X_train_ros['sex'] = X_train_ros['sex']  # já está incluso
y_split = y_group.str.split('_', expand=True)
X_train_ros['Sex'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)                # target original: income

# Verifica resultado
print(y_income.value_counts())
print(X_train_ros['Sex'].value_counts())


1
0    14628
1    14628
Name: count, dtype: int64
Sex
1    14628
0    14628
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_ros, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.709391  0.779152   0.327516  0.737646  0.461176        Random Forest
1   0.719966  0.756184   0.333593  0.734635  0.462953  Logistic Regression
2   0.716723  0.777385   0.333713  0.741293  0.466967    Gradient Boosting
3   0.736746  0.626325   0.329308  0.692022  0.431659              XGBoost
4   0.737310  0.675795   0.338346  0.712394  0.450928             CatBoost
5   0.716159  0.759717   0.330642  0.733802  0.460755     Ridge Classifier
6   0.701213  0.757067   0.317290  0.723836  0.447169       SGD Classifier
7   0.709673  0.763251   0.325424  0.731374  0.456298                  SVC
8   0.694726  0.608657   0.285774  0.659866  0.388936           KNeighbors
9   0.745629  0.643993   0.342254  0.704463  0.446965           GaussianNB
10  0.731246  0.649293   0.327540  0.698053  0.435427        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 7092 rows 22 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 7092 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e55457545e0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0541, mean = 0.407, max = 0.927
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.927, mean = -0.247, max = 0.938
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.Sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV     FPR       STP
0  1.020752  0.992978  0.938053  1.0301  1.010582
Métricas de viés detectadas para Random Forest: TPR=1.020752, ACC=0.992978, PPV=0.938053, FPR=1.0301, STP=1.010582
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  0.981651  0.979396  0.900568  1.053763  1.011111
Métricas de viés detectadas para Logist

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 8 -  NearMiss (remove exemplos da classe majoritária de forma informada, ou seja, não aleatoriamente, mas baseada na distância entre os exemplos da classe majoritária e os da classe minoritária.)

In [ ]:
from imblearn.under_sampling import NearMiss

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(["Diabetes_binary"], axis=1),
                                                    diabetes.Diabetes_binary,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['Sex'].astype(str) + "_" + y_train.astype(str)

near = NearMiss()
X_train_near, y_group = near.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_near['Sex'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_near['Sex'].value_counts())


1
0    2624
1    2624
Name: count, dtype: int64
Sex
0    2624
1    2624
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_near, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.445713  0.780919   0.193563  0.581483  0.310230        Random Forest
1   0.526650  0.783569   0.221805  0.630711  0.345742  Logistic Regression
2   0.412719  0.802120   0.187255  0.570439  0.303628    Gradient Boosting
3   0.397208  0.809187   0.184121  0.564074  0.299984              XGBoost
4   0.408206  0.800353   0.185770  0.567039  0.301548             CatBoost
5   0.648618  0.750883   0.277778  0.690039  0.405534     Ridge Classifier
6   0.530457  0.756184   0.218926  0.621884  0.339548       SGD Classifier
7   0.464467  0.778269   0.198961  0.591567  0.316906                  SVC
8   0.568105  0.704947   0.226255  0.623531  0.342563           KNeighbors
9   0.534687  0.675795   0.206869  0.591840  0.316770           GaussianNB
10  0.409334  0.811837   0.187743  0.572362  0.304961        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 7092 rows 22 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 7092 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e55457545e0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.108, mean = 0.622, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = -0.462, max = 0.88
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data           

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.Sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV      FPR       STP
0  1.053947  0.993289  0.964467  1.00813  1.014085
Métricas de viés detectadas para Random Forest: TPR=1.053947, ACC=0.993289, PPV=0.964467, FPR=1.00813, STP=1.014085
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV      FPR       STP
0  1.047059  0.951852  0.918103  1.06746  1.056569
Métricas de viés detectadas para Logis

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 9 - BorderlineSMOTE (Para superar a deficiência do SMOTE, ele identifica dois conjuntos de pontos — Ruído e Fronteira)

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(["Diabetes_binary"], axis=1),
                                                    diabetes.Diabetes_binary,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['Sex'].astype(str) + "_" + y_train.astype(str)

border = BorderlineSMOTE(random_state = 42, kind = 'borderline-2')
X_train_border, y_group = border.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_border['Sex'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_border['Sex'].value_counts())


1
0    14628
1    14628
Name: count, dtype: int64
Sex
1    14628
0    14628
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_border, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.787084  0.555654   0.384474  0.693347  0.454480        Random Forest
1   0.729836  0.712014   0.336394  0.722618  0.456916  Logistic Regression
2   0.816836  0.433746   0.427328  0.661671  0.430513    Gradient Boosting
3   0.829667  0.285336   0.447368  0.609195  0.348436              XGBoost
4   0.833333  0.282686   0.463768  0.610302  0.351262             CatBoost
5   0.730400  0.704947   0.335859  0.720091  0.454960     Ridge Classifier
6   0.711647  0.714664   0.319637  0.712869  0.441714       SGD Classifier
7   0.773971  0.611307   0.373046  0.708087  0.463341                  SVC
8   0.721658  0.537986   0.295631  0.647265  0.381579           KNeighbors
9   0.749718  0.606007   0.340447  0.691510  0.435971           GaussianNB
10  0.777073  0.477032   0.353172  0.655546  0.405862        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 7092 rows 22 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 7092 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e55457545e0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0271, mean = 0.332, max = 0.866
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.865, mean = -0.173, max = 0.961
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.Sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  1.059369  0.983627  0.894608  1.167742  1.100457
Métricas de viés detectadas para Random Forest: TPR=1.059369, ACC=0.983627, PPV=0.894608, FPR=1.167742, STP=1.100457
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
0  1.047482  0.957162  0.867769  1.185185  1.119497
Métricas de viés detectadas para 

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 10 - Reweighting ( Reponderação sensível de classe) - Sample Weight


In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(diabetes.drop(["Diabetes_binary"], axis=1),
                                                    diabetes.Diabetes_binary,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
sex_train = X_train['Sex']  # ou como for sua variável

# Rebalancear o sexo no treino usando sample_weight
sex_counts = sex_train.value_counts(normalize=True)
sex_weights = {sex: 1.0 / freq for sex, freq in sex_counts.items()}

# Cria o vetor de pesos
sample_weight = sex_train.map(sex_weights).values

#### Treinando para vários algoritmos

In [ ]:
model_pipelines = {
    'Random Forest': clf_forest,
    'Logistic Regression': clf_logreg,
    'Gradient Boosting': clf_gb,
    'XGBoost': clf_xgb,
    #'LightGBM': clf_lgbm,
    'CatBoost': clf_catboost,
    'Ridge Classifier': clf_ridge,
    'SGD Classifier': clf_sgd,
    #'SVC': clf_svc,
    #'KNeighbors': clf_knn,
    #'GaussianNB': clf_nb,
    #'MLPClassifier': clf_mlp
}

In [ ]:
trained_models = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier


In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train, classifier__sample_weight=sample_weight)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


   accuracy    recall  precision       auc        f1                model
0  0.846024  0.086572   0.628205  0.538420  0.152174        Random Forest
1  0.840948  0.188163   0.504739  0.576548  0.274131  Logistic Regression
2  0.846024  0.196113   0.549505  0.582788  0.289062    Gradient Boosting
3  0.834884  0.234099   0.465729  0.591546  0.311581              XGBoost
4  0.839255  0.198763   0.491266  0.579835  0.283019             CatBoost
5  0.845037  0.084806   0.603774  0.537118  0.148722     Ridge Classifier
6  0.827834  0.196113   0.416510  0.571966  0.266667       SGD Classifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 7092 rows 22 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 7092 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7e55457545e0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0131, mean = 0.169, max = 0.724
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.724, mean = -0.00916, max = 0.985
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data   

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.Sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=1)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: No bias was detected!

Conclusion: your model is fair in terms of checked fairness criteria.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC     PPV  FPR       STP
0  1.135802  1.015495  1.0096  1.0  1.095238
Métricas de viés detectadas para Random Forest: TPR=1.135802, ACC=1.015495, PPV=1.0096, FPR=1.0, STP=1.095238
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 1 metric: FPR

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot be automatically approved based on these metrics.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
    

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



# Resultados Finais

In [ ]:
all_performance_results.to_excel(r"Performance dos algoritmos sem balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento.to_excel(r"Performance dos algoritmos com 1º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_dois.to_excel(r"Performance dos algoritmos com 2º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_tres.to_excel(r"Performance dos algoritmos com 3º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_quatro.to_excel(r"Performance dos algoritmos com 4º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_fairness_checks.to_excel(r"Resultado da avaliação se os modelos são justos - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento.to_excel(r"Resultado da avaliação se os modelos são justos com 1º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_dois.to_excel(r"Resultado da avaliação se os modelos são justos com 2º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_tres.to_excel(r"Resultado da avaliação se os modelos são justos com 3º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_quatro.to_excel(r"Resultado da avaliação se os modelos são justos com 4º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)